In [2]:
import h2o
import numpy as np
import math
import warnings
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC,LinearSVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/jupyter/.*')
h2o.init(nthreads = -1)

telco_dataset = h2o.import_file("https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv")

from sklearn.preprocessing import quantile_transform

df = telco_dataset.as_data_frame()
df['MonthlyCharges'] = pd.DataFrame(np\
             .ceil(quantile_transform(df['MonthlyCharges'].fillna(0)\
                                      .values\
                                      .reshape(-1, 1), n_quantiles=10, random_state=0, copy=True
                                     )/0.1
                  ))[0].apply(lambda x: 'D{}'.format(int(x)))
df['TotalCharges'] = pd.DataFrame(np\
             .ceil(quantile_transform(df['TotalCharges'].fillna(0)\
                                      .values\
                                      .reshape(-1, 1), n_quantiles=10, random_state=0, copy=True
                                     )/0.1
                  ))[0].apply(lambda x: 'D{}'.format(int(x)))

df['tenure'] = pd.DataFrame(np\
             .ceil(quantile_transform(df['tenure'].fillna(0)\
                                      .values\
                                      .reshape(-1, 1), n_quantiles=10, random_state=0, copy=True
                                     )/0.1
                  ))[0].apply(lambda x: 'D{}'.format(int(x)))

bin_col = ['Dependents'
           ,'Partner'
           ,'PhoneService'
           ,'MultipleLines'
           ,'OnlineSecurity'
           ,'OnlineBackup'
           ,'DeviceProtection'
           ,'TechSupport'
           ,'StreamingTV'
           ,'StreamingMovies'
           ,'PaperlessBilling'
          ]

for col in bin_col :
    df[col] = df[col].apply(lambda x: 1 if x.upper() == 'YES' else 0)
    
X = pd.get_dummies(df[['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod' ,'MonthlyCharges','TotalCharges', 'tenure']]).fillna(0)
    
y = df['Churn'].values.reshape(-1 , 1)
print(X.shape)
print(y.shape)


X_scale = StandardScaler().fit_transform(X)

#X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.2, random_state=101)

model = LogisticRegression()
result = model.fit(X_scale, y)
prediction_test = model.predict(X_scale)
print (metrics.accuracy_score(y, prediction_test))

weights = pd.Series(model.coef_[0],
 index=X.columns.values)

print('Top 5 positive correlation \n')
print(weights.sort_values(ascending = False).head(5))


print('Top 5 negative correlation \n')
print(weights.sort_values(ascending = True).head(5))

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 min 21 secs
H2O cluster timezone:,Asia/Singapore
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_user_i18ihg
H2O cluster total nodes:,1
H2O cluster free memory:,863 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
(7043, 54)
(7043, 1)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8087462728950732
Top 5 positive correlation 

InternetService_Fiber optic    0.616338
tenure_D1                      0.429805
Contract_Month-to-month        0.339535
MonthlyCharges_D1              0.273627
tenure_D2                      0.250072
dtype: float64
Top 5 negative correlation 

InternetService_No   -0.714918
Contract_Two year    -0.381646
tenure_D10           -0.310922
tenure_D9            -0.191812
MonthlyCharges_D8    -0.175374
dtype: float64


In [ ]:
# For the result we found that Customer with fiber optic connection having the highest churn rate. 
# This probably due to services issues. Using this Analysis we could highlight this issue so that a more comprehansive
# investiogation are done to avoid more customer churn.
# Others factor we can see from this analysis is that, new customer have higher churn rate compare to customer which been
# with the telco from much longer period of time ( tenure in decile 1 & 2 compare to tenure in decile 10 & 9)